## 데이터셋의 토큰 길이를 체크하기 위한 노트북

### **1. Imports**

In [1]:
import sys
sys.path.append("../")  ## utils 함수 이용을 위한 패스 설정

import pandas as pd
import datasets
from utils import remove_hangul
from transformers import AutoTokenizer

import matplotlib.pyplot as plt ## 히스토그램 뿌리기

/root/anaconda3/envs/LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **2. 데이터셋 로드**

`-` 데이터셋과 시스템 프롬프트를 로드한 다음, 한글 및 불필요한 컬럼을 제거합니다.

In [4]:
ds = datasets.load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned")

In [ ]:
ds.remove_columns([])

DatasetDict({
    train: Dataset({
        features: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model'],
        num_rows: 60917
    })
})

In [ ]:
## 유저 프롬프트
df = pd.read_csv("data/data_all.csv", encoding = "utf-8")

## 시스템 프롬프트
# with open("../data/system_prompt.txt", "r") as f:
#     system_prompt = f.read()

## datasets.Dataset 객체로 변환
ds = datasets.Dataset.from_pandas(df)
columns_to_remove = [f for f in list(ds.features) if f not in ["subject_id", "text"]]

## Explicit format
train_ds = ds.map(
    lambda sample: {
        "text": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": sample["text"]}
            ]
    }
)

## 한글 제거 및 불필요한 컬럼 제거
train_ds = train_ds.map(lambda sample: remove_hangul(sample, column = "text"))
train_ds = train_ds.map(remove_columns = columns_to_remove, batched = False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/data_all.csv'

### **3. 토크나이저 로드 및 토크나이징**

`-` `Llama-3.1-8B-Instruct` 모델의 토크나이저를 로드한 뒤, 토큰화를 수행합니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    use_fast = True,
    trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

LLAMA_3_CHAT_TEMPLATE = (
    "{{ bos_token }}"
    "{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
            "{{ '<|start_header_id|>system<|end_header_id|>\n\n' + message['content'] + eos_token }}"
        "{% elif message['role'] == 'user' %}"
            "{{ '<|start_header_id|>user<|end_header_id|>\n\n' + message['content'] +  eos_token }}"
        "{% elif message['role'] == 'assistant' %}"
            "{{ '<|start_header_id|>assistant<|end_header_id|>\n\n'}}"
            "{% generation %}"
            "{{ message['content'] +  eos_token }}"
            "{% endgeneration %}"
        "{% endif %}"
    "{% endfor %}"
    "{%- if add_generation_prompt %}"
    "{{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}"
    "{%- endif %}"
)

tokenizer.chat_template = LLAMA_3_CHAT_TEMPLATE

def template_dataset(example):
    """
    데이터셋 객체에 매핑시키는 함수. 
    """
    return {"token_size": len(tokenizer.apply_chat_template(example["text"], tokenize = True))}

`-` 전체 데이터셋의 토큰 길이 파악

* 훈련 및 생성 작업에 참고할 수 있도록 토큰 길이를 파악합니다.

In [ ]:
train_ds = datasets.load_dataset("json", "../data/data_all.json", split = "train")
train_ds = train_ds.map(template_dataset)

print(f"max token_size: {max(train_ds['token_size'])}")
print(f"min token_size: {min(train_ds['token_size'])}")

Map: 100%|██████████| 145862/145862 [10:38<00:00, 228.30 examples/s]


max token_size: 17176
min token_size: 230


### **4. 훈련 데이터셋의 토큰 길이 파악**

`-` Truncation을 위해서 훈련 데이터셋만의 토큰 길이도 파악할 필요가 있습니다.

* 입력 텍스트의 토큰 길이가 길어질수록 VRAM 사용량 또한 산술적으로 증가하므로, 이를 적당히 조율합니다.
* 분포를 파악해서 큰 영향이 없도록 최대 토큰 사이즈를 조정합니다.

In [11]:
sft_ds = datasets.load_dataset("json", data_files = "../data/sft_train_dataset.json", split = "train")
dpo_ds = datasets.load_dataset("json", data_files = "../data/dpo_train_dataset.json", split = "train")

sft_ds = sft_ds.map(
    lambda example:
        {"token_size": len(tokenizer.apply_chat_template(example["messages"][:2], tokenize = True))}
)
dpo_ds = dpo_ds.map(
    lambda example:
        {"token_size": len(tokenizer.apply_chat_template(example["prompt"], tokenize = True))}
)

Generating train split: 90 examples [00:00, 4195.28 examples/s]
Generating train split: 1781 examples [00:00, 11776.92 examples/s]
Map: 100%|██████████| 1781/1781 [00:11<00:00, 149.77 examples/s]


In [ ]:
pd.DataFrame(train_ds["token_size"]).to_csv("all_token_length.csv", encoding = "utf-8")

AttributeError: 'Column' object has no attribute 'dtype'